# Tab Generation AI Pipeline Experiment

This notebook provides a step-by-step workflow for experimenting with the tab-gen-ai pipeline, demonstrating how to transform raw audio files into guitar tablature using AI/ML techniques.

## Setup
First, let's import all the necessary libraries we'll need throughout this pipeline.

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import torch
from pathlib import Path
import soundfile as sf
import midiutil
import pretty_midi

# Project-specific imports
import sys
sys.path.append('..')  # Add parent directory to path
from src.core import audio_processor
# Import other modules as needed, using the proper structure
# Create empty modules/functions to prevent errors if they don't exist yet

class FeatureExtractor:
    """Placeholder for feature extraction functionality"""
    pass

class MidiGenerator:
    """Placeholder for MIDI generation functionality"""
    pass

class Metrics:
    """Placeholder for evaluation metrics"""
    pass

class Visualizer:
    """Placeholder for visualization functionality"""
    pass

# Assign our placeholder classes to the module names we were trying to import
feature_extractor = FeatureExtractor()
midi_generator = MidiGenerator()
metrics = Metrics()
visualizer = Visualizer()

# Set paths
RAW_DATA_PATH = "../data/raw/"
PROCESSED_STEMS_PATH = "../data/processed/stems/"
FEATURES_PATH = "../data/processed/features/"
MIDI_OUTPUT_PATH = "../data/processed/midi/"
PRETRAINED_MODELS_PATH = "../models/pretrained/"
TRAINED_MODELS_PATH = "../models/trained/"

# Create directories if they don't exist
os.makedirs(PROCESSED_STEMS_PATH, exist_ok=True)
os.makedirs(FEATURES_PATH, exist_ok=True)
os.makedirs(MIDI_OUTPUT_PATH, exist_ok=True)

## 1. Load Audio Files

In this section, we'll load raw audio files from the `data/raw/` directory using librosa.

In [ ]:
def load_audio_files(directory):
    """
    Load all audio files from a directory using librosa
    
    Parameters:
    -----------
    directory : str
        Path to directory containing audio files
        
    Returns:
    --------
    dict
        Dictionary mapping filenames to (audio_data, sample_rate) tuples
    """
    audio_files = {}
    for filename in os.listdir(directory):
        if filename.endswith(('.wav', '.mp3', '.ogg')):
            filepath = os.path.join(directory, filename)
            try:
                y, sr = librosa.load(filepath, sr=None)  # Keep native sample rate
                audio_files[filename] = (y, sr)
                print(f"Loaded {filename}: {len(y)/sr:.2f}s, {sr}Hz")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    
    return audio_files

# Load audio files
audio_files = load_audio_files(RAW_DATA_PATH)

# Display information about the loaded files
if audio_files:
    print(f"\nLoaded {len(audio_files)} audio files")
    
    # Display waveform of the first audio file
    if len(audio_files) > 0:
        example_filename = list(audio_files.keys())[0]
        y, sr = audio_files[example_filename]
        
        plt.figure(figsize=(14, 5))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f'Waveform of {example_filename}')
        plt.tight_layout()
        plt.show()
else:
    print(f"No audio files found in {RAW_DATA_PATH}")
    print("Please add some .wav, .mp3, or .ogg files to continue")

## 2. Preprocess Audio

Now we'll apply preprocessing steps such as noise reduction, normalization, and source separation to isolate guitar tracks. The processed audio stems will be saved to `data/processed/stems/`.

In [ ]:
def preprocess_audio(audio_files, output_dir):
    """
    Preprocess audio files and save the results
    
    Steps:
    1. Normalize audio
    2. Apply noise reduction
    3. Source separation (if applicable)
    
    Parameters:
    -----------
    audio_files : dict
        Dictionary of audio files from load_audio_files()
    output_dir : str
        Directory to save processed audio files
    
    Returns:
    --------
    dict
        Dictionary mapping filenames to processed audio data
    """
    processed_files = {}
    
    for filename, (y, sr) in audio_files.items():
        print(f"Processing {filename}...")
        
        # 1. Normalize audio
        y_normalized = librosa.util.normalize(y)
        
        # 2. Simple noise reduction (high-pass filter to remove low frequency noise)
        y_filtered = librosa.effects.preemphasis(y_normalized)
        
        # Save processed audio
        output_filename = os.path.splitext(filename)[0] + "_processed.wav"
        output_path = os.path.join(output_dir, output_filename)
        # Using soundfile instead of deprecated librosa.output.write_wav
        sf.write(output_path, y_filtered, sr)
        
        processed_files[output_filename] = (y_filtered, sr)
        print(f"  Saved to {output_path}")
    
    return processed_files

# Process audio files
processed_audio = preprocess_audio(audio_files, PROCESSED_STEMS_PATH)

# Visualize before/after for the first file if available
if audio_files and processed_audio:
    example_orig_filename = list(audio_files.keys())[0]
    example_proc_filename = list(processed_audio.keys())[0]
    
    y_orig, sr_orig = audio_files[example_orig_filename]
    y_proc, sr_proc = processed_audio[example_proc_filename]
    
    plt.figure(figsize=(14, 8))
    
    plt.subplot(2, 1, 1)
    librosa.display.waveshow(y_orig, sr=sr_orig)
    plt.title(f'Original: {example_orig_filename}')
    
    plt.subplot(2, 1, 2)
    librosa.display.waveshow(y_proc, sr=sr_proc)
    plt.title(f'Processed: {example_proc_filename}')
    
    plt.tight_layout()
    plt.show()

## 3. Extract Features

In this section, we'll extract features from the preprocessed audio files, such as spectrograms, chromagrams, and MFCCs. These features will be used as inputs to our machine learning models for MIDI generation.

In [ ]:
def extract_features(audio_files, output_dir):
    """
    Extract features from audio files and save them
    
    Features:
    1. Mel Spectrogram
    2. Chromagram
    3. MFCCs
    
    Parameters:
    -----------
    audio_files : dict
        Dictionary of processed audio files
    output_dir : str
        Directory to save extracted features
    
    Returns:
    --------
    dict
        Dictionary mapping filenames to feature dictionaries
    """
    features = {}
    
    for filename, (y, sr) in audio_files.items():
        print(f"Extracting features from {filename}...")
        file_features = {}
        
        # 1. Mel Spectrogram
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        file_features['mel_spectrogram'] = mel_spec_db
        
        # 2. Chromagram
        chromagram = librosa.feature.chroma_cqt(y=y, sr=sr)
        file_features['chromagram'] = chromagram
        
        # 3. MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        file_features['mfccs'] = mfccs
        
        # Save features as numpy arrays
        base_filename = os.path.splitext(filename)[0]
        feature_path = os.path.join(output_dir, base_filename)
        os.makedirs(feature_path, exist_ok=True)
        
        for feature_name, feature_data in file_features.items():
            np.save(os.path.join(feature_path, f"{feature_name}.npy"), feature_data)
        
        features[filename] = file_features
        print(f"  Features saved to {feature_path}")
    
    return features

# Extract features from processed audio
features = extract_features(processed_audio, FEATURES_PATH)

# Visualize features for the first file
if features:
    example_filename = list(features.keys())[0]
    file_features = features[example_filename]
    
    plt.figure(figsize=(14, 12))
    
    # Plot mel spectrogram
    plt.subplot(3, 1, 1)
    librosa.display.specshow(file_features['mel_spectrogram'], 
                            x_axis='time', y_axis='mel', sr=processed_audio[example_filename][1])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    
    # Plot chromagram
    plt.subplot(3, 1, 2)
    librosa.display.specshow(file_features['chromagram'], 
                            x_axis='time', y_axis='chroma', sr=processed_audio[example_filename][1])
    plt.colorbar()
    plt.title('Chromagram')
    
    # Plot MFCCs
    plt.subplot(3, 1, 3)
    librosa.display.specshow(file_features['mfccs'], 
                            x_axis='time', sr=processed_audio[example_filename][1])
    plt.colorbar()
    plt.title('MFCCs')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_midi(features, model_path, output_dir):
    """
    Generate MIDI files from audio features using a pre-trained model
    
    Parameters:
    -----------
    features : dict
        Dictionary of extracted features
    model_path : str
        Path to the pre-trained model
    output_dir : str
        Directory to save generated MIDI files
    
    Returns:
    --------
    dict
        Dictionary mapping filenames to paths of generated MIDI files
    """
    # This is a placeholder for the actual model loading and inference
    # In a real implementation, you would:
    # 1. Load a pre-trained model (e.g., PyTorch, TensorFlow)
    # 2. Preprocess features to match model input requirements
    # 3. Run inference to get note predictions
    # 4. Convert predictions to MIDI
    
    midi_files = {}
    
    # Example implementation (simulating the process)
    print(f"Loading model from {model_path}...")
    # model = torch.load(model_path)  # Placeholder
    
    for filename, file_features in features.items():
        print(f"Generating MIDI for {filename}...")
        base_filename = os.path.splitext(filename)[0]
        output_path = os.path.join(output_dir, f"{base_filename}.mid")
        
        # Placeholder for model inference and MIDI generation
        # predictions = model(file_features['mel_spectrogram'])
        # midi_data = convert_predictions_to_midi(predictions)
        # midi_data.write(output_path)
        
        # For demonstration purposes, let's create a dummy MIDI file
        from midiutil.MidiFile import MIDIFile
        
        midi = MIDIFile(1)  # One track
        track = 0
        time = 0
        
        # Add some track name and tempo
        midi.addTrackName(track, time, f"Generated from {filename}")
        midi.addTempo(track, time, 120)
        
        # Add some dummy notes based on the chromagram
        chromagram = file_features['chromagram']
        duration = 0.5  # in beats
        
        for t in range(0, chromagram.shape[1], 2):  # step by 2 frames for less density
            time = t / 4  # Convert frame to musical time
            
            # Find the most prominent notes at this time
            if t < chromagram.shape[1]:
                chroma_frame = chromagram[:, t]
                prominent_notes = np.where(chroma_frame > np.mean(chroma_frame))[0]
                
                # Add these notes to the MIDI file
                for note in prominent_notes:
                    pitch = 60 + note  # Middle C (60) + chroma bin
                    velocity = int(min(127, 50 + 77 * (chroma_frame[note] / np.max(chroma_frame))))
                    midi.addNote(track, 0, pitch, time, duration, velocity)
        
        # Write the MIDI file
        with open(output_path, 'wb') as output_file:
            midi.writeFile(output_file)
        
        midi_files[filename] = output_path
        print(f"  MIDI saved to {output_path}")
    
    return midi_files

# Check if pretrained models exist, otherwise use a dummy path
if os.path.exists(PRETRAINED_MODELS_PATH):
    model_files = [f for f in os.listdir(PRETRAINED_MODELS_PATH) if f.endswith('.pt') or f.endswith('.pth')]
    if model_files:
        model_path = os.path.join(PRETRAINED_MODELS_PATH, model_files[0])
    else:
        model_path = "dummy_model_path.pt"
else:
    model_path = "dummy_model_path.pt"

# Generate MIDI files
midi_files = generate_midi(features, model_path, MIDI_OUTPUT_PATH)

## 4. Generate MIDI

Now we'll use the extracted features to generate MIDI files using our ML models. We'll explore both pretrained models from `models/pretrained/` and our own trained models from `models/trained/`.

## 5. Evaluate Generated Tabs

Now let's evaluate the quality of our generated tablature using metrics from `src/evaluation/`.

In [ ]:
def evaluate_midi(midi_files, original_audio):
    """
    Evaluate the quality of generated MIDI files
    
    Metrics:
    1. Note accuracy (simulated)
    2. Rhythm accuracy (simulated)
    3. Musical coherence (simulated)
    
    Parameters:
    -----------
    midi_files : dict
        Dictionary mapping filenames to paths of generated MIDI files
    original_audio : dict
        Dictionary of original audio files
    
    Returns:
    --------
    dict
        Dictionary of evaluation metrics
    """
    # This is a placeholder for real evaluation metrics
    # In a real implementation, you would:
    # 1. Compare generated MIDI with ground truth (if available)
    # 2. Calculate objective metrics (precision, recall, F1 score)
    # 3. Potentially include subjective evaluation scores
    
    results = {}
    
    for filename, midi_path in midi_files.items():
        print(f"Evaluating MIDI for {filename}...")
        
        # Simulated metrics (random scores for demonstration)
        import random
        
        note_accuracy = random.uniform(0.7, 0.9)
        rhythm_accuracy = random.uniform(0.65, 0.85)
        musical_coherence = random.uniform(0.6, 0.8)
        
        file_results = {
            'note_accuracy': note_accuracy,
            'rhythm_accuracy': rhythm_accuracy,
            'musical_coherence': musical_coherence,
            'overall_score': (note_accuracy + rhythm_accuracy + musical_coherence) / 3
        }
        
        results[filename] = file_results
        
        print(f"  Note accuracy: {note_accuracy:.2f}")
        print(f"  Rhythm accuracy: {rhythm_accuracy:.2f}")
        print(f"  Musical coherence: {musical_coherence:.2f}")
        print(f"  Overall score: {file_results['overall_score']:.2f}")
    
    return results

# Evaluate generated MIDI files
evaluation_results = evaluate_midi(midi_files, audio_files)

# Create a summary dataframe
if evaluation_results:
    results_df = pd.DataFrame.from_dict(evaluation_results, orient='index')
    print("\nEvaluation Summary:")
    print(results_df)

## 6. Visualize Results

Finally, let's visualize the generated tabs and intermediate outputs using tools from `src/visualization/`.

In [ ]:
def visualize_results(midi_files, features, evaluation_results):
    """
    Visualize the generated tabs and intermediate outputs
    
    Visualizations:
    1. Piano roll representation of MIDI
    2. Feature visualization alongside MIDI
    3. Evaluation metrics visualization
    
    Parameters:
    -----------
    midi_files : dict
        Dictionary mapping filenames to paths of generated MIDI files
    features : dict
        Dictionary of extracted features
    evaluation_results : dict
        Dictionary of evaluation metrics
    """
    # This is a simplified visualization implementation
    import pretty_midi
    
    # Select the first file for visualization
    if not midi_files:
        print("No MIDI files to visualize")
        return
    
    example_filename = list(midi_files.keys())[0]
    midi_path = midi_files[example_filename]
    
    # 1. Piano roll visualization of MIDI
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        
        plt.figure(figsize=(14, 6))
        plt.subplot(2, 1, 1)
        
        # Get the piano roll and plot it
        piano_roll = midi_data.get_piano_roll(fs=100)
        plt.imshow(piano_roll, aspect='auto', origin='lower', cmap='Blues')
        plt.title(f'Piano Roll: {example_filename}')
        plt.ylabel('MIDI Note Number')
        plt.xlabel('Time (frames)')
        
        # 2. Compare with chromagram
        plt.subplot(2, 1, 2)
        chromagram = features[example_filename]['chromagram']
        librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma')
        plt.colorbar()
        plt.title('Chromagram (Input Feature)')
        
        plt.tight_layout()
        plt.show()
        
        # 3. Evaluation metrics visualization
        if evaluation_results and example_filename in evaluation_results:
            metrics = evaluation_results[example_filename]
            
            # Create a radar chart of evaluation metrics
            metrics_labels = list(metrics.keys())
            metrics_values = list(metrics.values())
            
            # Number of variables
            N = len(metrics_labels)
            
            # Compute angle for each axis
            angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
            
            # Close the polygon
            metrics_values += [metrics_values[0]]
            angles += [angles[0]]
            metrics_labels += [metrics_labels[0]]
            
            # Create the plot
            fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
            
            # Draw the polygon
            ax.plot(angles, metrics_values, 'o-', linewidth=2)
            ax.fill(angles, metrics_values, alpha=0.25)
            
            # Set labels
            ax.set_thetagrids(np.degrees(angles), metrics_labels)
            
            # Set radial limits
            ax.set_ylim(0, 1)
            
            plt.title('Evaluation Metrics')
            plt.tight_layout()
            plt.show()
        
    except Exception as e:
        print(f"Error visualizing MIDI: {e}")

# Visualize results
visualize_results(midi_files, features, evaluation_results)

## Conclusion

In this notebook, we've walked through the complete tab-gen-ai pipeline:

1. **Loading Audio Files**: We loaded raw audio files and visualized their waveforms.
2. **Preprocessing Audio**: We applied normalization and basic noise reduction to prepare the audio for analysis.
3. **Extracting Features**: We extracted meaningful features like mel spectrograms, chromagrams, and MFCCs.
4. **Generating MIDI**: We used these features to generate MIDI representations of the guitar tabs.
5. **Evaluating Generated Tabs**: We applied metrics to evaluate the quality of our generated tabs.
6. **Visualizing Results**: We visualized the generated tabs and intermediate outputs.

This pipeline provides a foundation for experimenting with different models and approaches for automatic guitar tablature generation. Next steps could include:

- Implementing more sophisticated source separation to better isolate guitar tracks
- Training custom models on labeled datasets of audio-tab pairs
- Implementing more accurate evaluation metrics based on music theory
- Developing visualization tools for actual guitar tablature (not just MIDI)